In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install keras-rl
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.13.1 

In [3]:
from __future__ import division
import gym
import atari_py
from PIL import Image
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate
from keras.optimizers import Adam
from keras.utils import plot_model
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

#print(atari_py.list_games())

print(keras.__version__)
print(tf.__version__)


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

2.2.4
1.13.1


In [0]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('float32')/255 

    #def process_reward(self, reward):
     #   return np.clip(reward, -1., 1.)

In [0]:
#game = 'BeamRider-v0' #9
#game = 'Breakout-v0' #4
#game = 'Solaris-v0' #18
#game = 'DemonAttack-v0' #6
#game = 'AirRaid-v0' #6

game1 = 'AirRaid-v0' #6
game2 = 'Centipede-v0' #18

env1 = gym.make(game1)
np.random.seed(123)
env1.seed(123)
env1_nb_actions = env1.action_space.n

env2 = gym.make(game2)
np.random.seed(123)
env2.seed(123)
env2_nb_actions = env2.action_space.n

total_actions = 18

In [6]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE


# CNN for environment 1
env1_input_layer = Input(shape=input_shape)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    env1_permute = Permute((2, 3, 1))(env1_input_layer)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    env1_permute = Permute((1, 2, 3))(env1_input_layer)
else:
    raise RuntimeError('Unknown image_dim_ordering for environment 1.')

env1_Conv_1 = Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu')(env1_permute)
env1_Conv_2 = Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu')(env1_Conv_1)
env1_Conv_3 = Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu')(env1_Conv_2)
env1_flatten = Flatten()(env1_Conv_3)
env1_dense = Dense(512, activation = 'relu')(env1_flatten)


# CNN for environment 2
env2_input_layer = Input(shape=input_shape)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    env2_permute = Permute((2, 3, 1))(env2_input_layer)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    env2_permute = Permute((1, 2, 3))(env2_input_layer)
else:
    raise RuntimeError('Unknown image_dim_ordering for environment 2.')
    
env2_Conv_1 = Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu')(env2_permute)
env2_Conv_2 = Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu')(env2_Conv_1)
env2_Conv_3 = Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu')(env2_Conv_2)
env2_flatten = Flatten()(env2_Conv_3)
env2_dense = Dense(512, activation = 'relu')(env2_flatten)

shared_layer = Dense(total_actions, activation = 'linear', name = 'shared_layer')    #Shared layer

env1_output = Dense(env1_nb_actions, activation = 'linear', name = 'env1_output')(shared_layer(env1_dense))
env2_output =  Dense(env2_nb_actions, activation = 'linear', name = 'env2_output')(shared_layer(env2_dense))


model1 = Model(inputs = [env1_input_layer], outputs = [env1_output]) 
model2 =  Model(inputs = [env2_input_layer], outputs = [env2_output])


print(model1.summary())
plot_model(model1)
print(model2.summary())
plot_model(model2)


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 84, 84)         0         
_________________________________________________________________
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dens

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000000)

dqn1 = DQNAgent(model=model1, nb_actions=env1_nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn1.compile(Adam(lr=.00025), metrics=['mae'])


In [0]:
# from keras.callbacks import Callback as KerasCallback
# class save_at_episode(KerasCallback):
#     def on_episode_end(self, episode, logs={}):
#         self.model.save_weights('drive/My Drive/Colab Notebooks/Game Data/Breakout_weights_v1_{}'.format(episode), overwrite=True)
#         #print(" saved episode:{}".format(episode))
        

In [7]:
dqn1.fit(env1, nb_steps=300000, visualize=False, verbose=1) #callbacks=[save_at_episode()]
dqn1.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_v2',overwrite=True)
#'/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights'

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 30s 3ms/step - reward: 0.7325
13 episodes - episode_reward: 534.615 [225.000, 950.000] - ale.lives: 0.999

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.7125
13 episodes - episode_reward: 542.308 [75.000, 1250.000] - ale.lives: 0.999

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.6725
17 episodes - episode_reward: 407.353 [100.000, 975.000] - ale.lives: 0.998

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.7075
12 episodes - episode_reward: 610.417 [150.000, 1200.000] - ale.lives: 0.999

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: 0.7500
13 episodes - episode_reward: 501.923 [250.000, 950.000] - ale.lives: 0.999

Interval 6 (50000 steps perf

In [8]:
dqn1.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_v2')
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 1175.000, steps: 3509
Episode 2: reward: 375.000, steps: 1998
Episode 3: reward: 1775.000, steps: 4657
Episode 4: reward: 1350.000, steps: 3566
Episode 5: reward: 1000.000, steps: 3615
Episode 6: reward: 600.000, steps: 3211
Episode 7: reward: 1375.000, steps: 3270
Episode 8: reward: 450.000, steps: 2267
Episode 9: reward: 775.000, steps: 2635
Episode 10: reward: 650.000, steps: 2742


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

shared_weights, shared_bias = model1.get_layer('shared_layer').weights
sns.distplot(K.eval(shared_weights).ravel())
plt.savefig('/content/drive/My Drive/Colab Notebooks/Game Data/shared_weights_before_v2.jpg')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000000)

dqn2 = DQNAgent(model=model2, nb_actions=env2_nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn2.compile(Adam(lr=.00025), metrics=['mae'])


In [11]:
dqn2.fit(env2, nb_steps=300000, visualize=False, verbose=1) #callbacks=[save_at_episode()]
dqn2.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model2_weights_v2',overwrite=True)

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 2.5046
9 episodes - episode_reward: 2729.333 [918.000, 3906.000] - ale.lives: 2.022

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 2.5588
11 episodes - episode_reward: 2248.091 [745.000, 8097.000] - ale.lives: 2.006

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 2.9687
9 episodes - episode_reward: 3219.556 [770.000, 6850.000] - ale.lives: 2.021

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.2356
11 episodes - episode_reward: 2208.000 [680.000, 3485.000] - ale.lives: 2.048

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 2.2771
9 episodes - episode_reward: 2477.889 [485.000, 4944.000] - ale.lives: 1.943

Interval 6 (50000 step

In [18]:
dqn2.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model2_weights_v2')
dqn2.test(env2, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 2099.000, steps: 1500
Episode 2: reward: 2727.000, steps: 1945
Episode 3: reward: 1798.000, steps: 1234
Episode 4: reward: 333.000, steps: 1131
Episode 5: reward: 1121.000, steps: 748
Episode 6: reward: 830.000, steps: 978
Episode 7: reward: 1443.000, steps: 1245
Episode 8: reward: 1998.000, steps: 1289
Episode 9: reward: 1641.000, steps: 956
Episode 10: reward: 349.000, steps: 1048


In [13]:
shared_weights, shared_bias = model1.get_layer('shared_layer').weights
sns.distplot(K.eval(shared_weights).ravel())
plt.savefig('/content/drive/My Drive/Colab Notebooks/Game Data/shared_weights_after_v2.jpg')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
dqn1.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_v2_changed',overwrite=True)

In [18]:
dqn1.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_v2_changed')
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 50.000, steps: 261
Episode 2: reward: 100.000, steps: 405
Episode 3: reward: 75.000, steps: 362
Episode 4: reward: 225.000, steps: 2242
Episode 5: reward: 0.000, steps: 394
Episode 6: reward: 25.000, steps: 382
Episode 7: reward: 125.000, steps: 1017
Episode 8: reward: 0.000, steps: 266
Episode 9: reward: 50.000, steps: 398
Episode 10: reward: 75.000, steps: 293


In [0]:
#env.close()